In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [2]:
with open('key.json') as x:
    credentials = json.load(x)

In [3]:
api_key = credentials['api_key']

***Importing cleaned data from another notebook to pass to  omdb api***

In [4]:
original=pd.read_csv(r'C:\Users\Thund\NSS_Data_Analytics\Projects\Capstone\MovieRemakeCapstone\data\og.csv')
reem=pd.read_csv(r'C:\Users\Thund\NSS_Data_Analytics\Projects\Capstone\MovieRemakeCapstone\data\reem.csv')
three=pd.read_csv(r'C:\Users\Thund\NSS_Data_Analytics\Projects\Capstone\MovieRemakeCapstone\data\three.csv')
four=pd.read_csv(r'C:\Users\Thund\NSS_Data_Analytics\Projects\Capstone\MovieRemakeCapstone\data\four.csv')
five=pd.read_csv(r'C:\Users\Thund\NSS_Data_Analytics\Projects\Capstone\MovieRemakeCapstone\data\five.csv')

In [5]:
def api_pass(x,y):
    return 'http://www.omdbapi.com/?apikey='+api_key+'&t='+x+'&y='+y


***Adding api friendly column***

In [6]:
original['api']=[x.replace(' ','+') for x in original['title']]
reem['api']=[x.replace(' ','+') for x in reem['title']]
three['api']=[x.replace(' ','+') for x in three['title']]
four['api']=[x.replace(' ','+') for x in four['title']]
five['api']=[x.replace(' ','+') for x in five['title']]

In [7]:
original

,Unnamed: 0,year,title,api
0,0,1963,13 Assassins,13+Assassins
1,1,1960,13 Ghosts,13+Ghosts
2,2,2005,13 Tzameti,13+Tzameti
3,3,2009,3 Idiots,3+Idiots
4,4,1957,3:10 to Yuma,3:10+to+Yuma
...,...,...,...,...
531,531,1977,The Yellow Handkerchief,The+Yellow+Handkerchief
532,532,1948,Yellow Sky,Yellow+Sky
533,533,1961,Yojimbo,Yojimbo
534,534,1968,"Yours, Mine and Ours","Yours,+Mine+and+Ours"


In [8]:
#For some reason, the year column in my three and five dataframe did not read in correctly. Converting it to a string for api.
three['new_year']=[str(x) for x in three['year']]
five['new_year']=[str(x) for x in five['year']]

***testing the api***

In [9]:
endpoint = 'http://www.omdbapi.com/?apikey='+api_key+'&i=tt0115433'
response = requests.get(endpoint)

In [10]:
res=response.json()

In [11]:
res

{'Title': '101 Dalmatians',
 'Year': '1996',
 'Rated': 'G',
 'Released': '27 Nov 1996',
 'Runtime': '103 min',
 'Genre': 'Adventure, Comedy, Crime',
 'Director': 'Stephen Herek',
 'Writer': 'Dodie Smith, John Hughes',
 'Actors': 'Glenn Close, Jeff Daniels, Joely Richardson',
 'Plot': 'An evil high-fashion designer plots to steal dalmatian puppies in order to make an extravagant fur coat but creates an extravagant mess instead.',
 'Language': 'English, Spanish',
 'Country': 'United States, United Kingdom',
 'Awards': 'Nominated for 1 BAFTA Award3 wins & 10 nominations total',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMGI2NTQ2N2EtNDExMS00OWM2LTkzZWItZGUzY2M2MDFjODk0XkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '5.7/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '39%'},
  {'Source': 'Metacritic', 'Value': '49/100'}],
 'Metascore': '49',
 'imdbRating': '5.7',
 'imdbVotes': '116,781',
 'imdbID': 'tt0115433',
 'Type': 'movi

In [12]:
df=pd.json_normalize(res)
df

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
0,101 Dalmatians,1996,G,27 Nov 1996,103 min,"Adventure, Comedy, Crime",Stephen Herek,"Dodie Smith, John Hughes","Glenn Close, Jeff Daniels, Joely Richardson",An evil high-fashion designer plots to steal d...,...,49,5.7,"116,781",tt0115433,movie,04 Jan 2016,"$136,189,294",N/A,N/A,True


***Practice unpacking rating dictionary correctly***

In [13]:
rates=res['Ratings']
rates

[{'Source': 'Internet Movie Database', 'Value': '5.7/10'},
 {'Source': 'Rotten Tomatoes', 'Value': '39%'},
 {'Source': 'Metacritic', 'Value': '49/100'}]

In [14]:
rates2=rates[1]
rates2
#rates3=rates2[1]
#rates3

{'Source': 'Rotten Tomatoes', 'Value': '39%'}

***Creating lists of attributes***

In [15]:
api=original['api']
year=original['year']

api_list=api_pass(api, year)

originals=[]

for x in api_list:
    endpoint = x
    response = requests.get(endpoint)
    if response.status_code != 200:
        print(f'Failed: {response.status_code}')
    else:
        res=response.json()
        originals.append(res)
       
            

In [16]:
originals=pd.json_normalize(originals)
originals

,Response,Error,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,totalSeasons
0,False,Movie not found!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,True,NaN,13 Ghosts,1960,Not Rated,01 Jul 1960,85 min,"Horror, Mystery",William Castle,Robb White,...,56,6.0,"7,015",tt0053559,movie,27 Jun 2014,"$3,270,000",N/A,N/A,NaN
2,False,Movie not found!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,True,NaN,3 Idiots,2009,PG-13,25 Dec 2009,170 min,"Comedy, Drama",Rajkumar Hirani,"Abhijat Joshi, Rajkumar Hirani, Vidhu Vinod Ch...",...,67,8.4,"429,262",tt1187043,movie,29 Nov 2016,"$6,532,874",N/A,N/A,NaN
4,True,NaN,3:10 to Yuma,1957,Not Rated,07 Aug 1957,92 min,"Drama, Thriller, Western",Delmer Daves,"Halsted Welles, Elmore Leonard",...,80,7.6,"21,342",tt0050086,movie,16 Apr 2012,"$4,033,000",N/A,N/A,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,True,NaN,The Yellow Handkerchief,1977,N/A,28 Apr 1978,109 min,"Comedy, Drama",Yôji Yamada,"Yôji Yamada, Yoshitaka Asama, Pete Hamill",...,N/A,7.3,"1,283",tt0076935,movie,N/A,N/A,N/A,N/A,NaN
532,True,NaN,Yellow Sky,1948,Approved,24 Dec 1948,98 min,"Crime, Drama, Western",William A. Wellman,"Lamar Trotti, W.R. Burnett",...,N/A,7.4,"5,637",tt0040978,movie,04 Sep 2016,"$5,600,000",N/A,N/A,NaN
533,True,NaN,Yojimbo,1961,Not Rated,13 Sep 1961,110 min,"Action, Drama, Thriller",Akira Kurosawa,"Akira Kurosawa, Ryûzô Kikushima",...,93,8.2,"130,004",tt0055630,movie,23 Mar 2017,"$46,808",N/A,N/A,NaN
534,True,NaN,"Yours, Mine and Ours",1968,Unrated,24 Apr 1968,111 min,"Comedy, Family",Melville Shavelson,"Melville Shavelson, Mort Lachman, Madelyn Davis",...,N/A,7.1,"9,712",tt0063829,movie,16 Sep 2008,N/A,N/A,N/A,NaN


originals['Ratings']

In [19]:
prod_companies=originals.groupby('Production')['Production'].count()
prod_companies

Production
Chloe Productions, Wiseau-Films, TPW Films      1
Evanstone Films Ltd.                            1
Hammer Films                                    1
Marvista Entertainment                          1
N/A                                           473
VCI Entertainment                               1
Name: Production, dtype: int64

print(original_df.shape)
print(no_title.shape)

In [20]:
api=reem['api']
year=reem['year']

api_list=api_pass(api, year)

reems=[]

for x in api_list:
    endpoint = x
    response = requests.get(endpoint)
    if response.status_code != 200:
        print(f'Failed: {response.status_code}')
    else:
        res=response.json()
        reems.append(res)

In [21]:
reems=pd.json_normalize(reems)
reems

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response,Error,totalSeasons
0,13 Assassins,2010,R,25 Sep 2010,141 min,"Action, Adventure, Drama",Takashi Miike,"Kaneo Ikegami, Shoichirou Ikemiya, Daisuke Tengan","Koji Yakusho, Takayuki Yamada, Yûsuke Iseya",A group of assassins come together for a suici...,...,"68,058",tt1436045,movie,07 Jan 2017,"$802,778",N/A,N/A,True,NaN,NaN
1,OCT31 - 8 - Thirteen Ghosts (2001),2001,N/A,12 Oct 2022,N/A,N/A,N/A,N/A,N/A,N/A,...,N/A,tt22971930,movie,N/A,N/A,N/A,N/A,True,NaN,NaN
2,13,2010,R,01 Mar 2010,91 min,"Crime, Drama, Thriller",Géla Babluani,"Géla Babluani, Greg Pruss","Sam Riley, Alice Barrett, Gaby Hoffmann",A naive young man assumes a dead man's identit...,...,"43,626",tt0798817,movie,14 Mar 2017,N/A,N/A,N/A,True,NaN,NaN
3,Nanban,2012,Not Rated,12 Jan 2012,188 min,"Comedy, Drama",S. Shankar,"Rajkumar Hirani, Abhijat Joshi, Madhan Karky","Joseph Vijay, Jiiva, Srikanth","10 years after leaving college, two best frien...",...,"16,567",tt2180477,movie,N/A,N/A,N/A,N/A,True,NaN,NaN
4,3:10 to Yuma,2007,R,07 Sep 2007,122 min,"Action, Crime, Drama",James Mangold,"Halsted Welles, Michael Brandt, Derek Haas","Russell Crowe, Christian Bale, Ben Foster",A small-time rancher agrees to hold a captured...,...,"329,033",tt0381849,movie,14 Dec 2015,"$53,606,916",N/A,N/A,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,The Yellow Handkerchief,2008,PG-13,19 Nov 2009,102 min,"Adventure, Drama, Romance",Udayan Prasad,"Pete Hamill, Erin Dignam","William Hurt, Maria Bello, Kristen Stewart",A road trip through Louisiana transforms three...,...,"8,854",tt0954990,movie,05 May 2019,"$318,623",N/A,N/A,True,NaN,NaN
532,The Jackals,1967,N/A,15 Nov 1967,96 min,"Adventure, Western",Robert D. Webb,"Lamar Trotti, Harold Medford, W.R. Burnett","Vincent Price, Diana Ivarson, Robert Gunner",Bad bank robber falls in love with granddaught...,...,304,tt0061832,movie,N/A,N/A,N/A,N/A,True,NaN,NaN
533,A Fistful of Dollars,1964,R,18 Jan 1967,99 min,"Action, Drama, Western",Sergio Leone,"Adriano Bolzoni, Mark Lowell, Víctor Andrés Ca...","Clint Eastwood, Gian Maria Volontè, Marianne Koch",A wandering gunfighter plays two rival familie...,...,"230,127",tt0058461,movie,22 Jun 2012,"$14,500,000",N/A,N/A,True,NaN,NaN
534,"Yours, Mine & Ours",2005,PG,23 Nov 2005,88 min,"Comedy, Family, Romance",Raja Gosnell,"Ron Burch, David Kidd, Melville Shavelson","Dennis Quaid, Rene Russo, Jerry O'Connell",A widowed Coast Guard Admiral and a widowed ha...,...,"30,063",tt0443295,movie,15 Jun 2011,"$53,412,862",N/A,N/A,True,NaN,NaN


In [27]:
prod_companies=reems.groupby('Production')['Production'].count()
prod_companies

Production
Concorde-New Horizons      1
N/A                      499
iDream Productions         1
Name: Production, dtype: int64

In [22]:
api=three['api']
year=three['new_year']

api_list=api_pass(api, year)

threes=[]

for x in api_list:
    endpoint = x
    response = requests.get(endpoint)
    if response.status_code != 200:
        print(f'Failed: {response.status_code}')
    else:
        res=response.json()
        threes.append(res)

In [23]:
threes=pd.json_normalize(threes)
threes

,Response,Error,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,...,Ratings,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website
0,False,Movie not found!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,False,Movie not found!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,True,NaN,Pyar Jhukta Nahin,1985,N/A,11 Jan 1985,135 min,"Family, Romance",Vijay Sadanah,"S.H. Bihari, Partho Mukerjee, Amir Shamji",...,"[{'Source': 'Internet Movie Database', 'Value'...",N/A,6.5,211,tt0087958,movie,N/A,N/A,N/A,N/A
3,True,NaN,Mann - Soul's Heart,1999,Not Rated,22 Jun 1999,160 min,"Drama, Romance",Indra Kumar,"Mildred Cram, Aatish Kapadia, Naushir Khatau",...,"[{'Source': 'Internet Movie Database', 'Value'...",N/A,6.1,"5,829",tt0206921,movie,09 Sep 2016,N/A,N/A,N/A
4,True,NaN,Annie,2014,PG,19 Dec 2014,118 min,"Comedy, Drama, Family",Will Gluck,"Will Gluck, Aline Brosh McKenna, Thomas Meehan",...,"[{'Source': 'Internet Movie Database', 'Value'...",33,5.3,"40,132",tt1823664,movie,10 Dec 2015,"$85,911,262",N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,True,NaN,Victor and Victoria,1957,N/A,N/A,107 min,Comedy,Karl Anton,"Curt J. Braun, Reinhold Schünzel",...,"[{'Source': 'Internet Movie Database', 'Value'...",N/A,6.2,31,tt0051165,movie,N/A,N/A,N/A,N/A
83,True,NaN,Veera Madakari,2009,Not Rated,20 Mar 2009,175 min,Action,Sudeep,"Sudeep, Vijayendra Prasad, Kaviraj",...,"[{'Source': 'Internet Movie Database', 'Value'...",N/A,6.5,389,tt2917382,movie,N/A,N/A,N/A,N/A
84,True,NaN,What Men Want,2019,R,08 Feb 2019,117 min,"Comedy, Fantasy, Romance",Adam Shankman,"Tina Gordon, Peter Huyck, Alex Gregory",...,"[{'Source': 'Internet Movie Database', 'Value'...",49,5.4,"31,016",tt7634968,movie,23 Apr 2019,"$54,611,903",N/A,N/A
85,True,NaN,The Women,2008,PG-13,12 Sep 2008,114 min,"Comedy, Drama",Diane English,"Diane English, Clare Boothe Luce, Anita Loos",...,"[{'Source': 'Internet Movie Database', 'Value'...",27,5.0,"22,763",tt0430770,movie,01 Nov 2010,"$26,902,075",N/A,N/A


In [24]:
api=four['api']
year=four['year']

api_list=api_pass(api, year)

fours=[]

for x in api_list:
    endpoint = x
    response = requests.get(endpoint)
    if response.status_code != 200:
        print(f'Failed: {response.status_code}')
    else:
        res=response.json()
        fours.append(res)

In [25]:
fours=pd.json_normalize(fours)
fours

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response,Error
0,The 39 Steps,2008,TV-PG,28 Dec 2008,90 min,"Adventure, Crime, Mystery",James Hawes,"John Buchan, Lizzie Mickery","Rupert Penry-Jones, Lydia Leonard, David Haig","Richard Hannay, a mining engineer on holiday f...",...,6.3,"2,370",tt1282016,movie,02 Mar 2010,N/A,Warner Home Video,N/A,True,NaN
1,Nee Bareda Kadambari,1985,N/A,N/A,N/A,Drama,B.S. Dwarakish,Chi Udayashankar,"Bhavya, Hema Chaudhari, Lohitashwa",N/A,...,N/A,20,tt1392207,movie,N/A,N/A,N/A,N/A,True,NaN
2,Brewster's Millions,1935,Approved,05 Apr 1935,80 min,"Comedy, Musical",Thornton Freeland,"Arthur Wimperis, Donovan Pedelty, Clifford Grey","Jack Buchanan, Lili Damita, Nancy O'Neil",Jack Brewster is a pennyless English lad who l...,...,7.3,73,tt0023841,movie,N/A,N/A,N/A,N/A,True,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Movie not found!
4,A Perfect Murder,1998,R,05 Jun 1998,107 min,"Crime, Drama, Thriller",Andrew Davis,"Frederick Knott, Patrick Smith Kelly","Michael Douglas, Gwyneth Paltrow, Viggo Mortensen",A wealthy Wall Street speculator discovers tha...,...,6.6,"91,545",tt0120787,movie,28 Dec 2008,"$67,638,368",N/A,N/A,True,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Movie not found!
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Movie not found!
7,Freaky Friday,2018,TV-G,10 Aug 2018,90 min,"Comedy, Fantasy, Musical",Steve Carr,"Bridget Carpenter, Mary Rodgers, Brian Yorkey","Cozi Zuehlsdorff, Heidi Blickenstaff, Jason Ma...",A musical about what happens when a mom and da...,...,3.9,"1,672",tt7631348,movie,13 Aug 2018,N/A,N/A,N/A,True,NaN
8,Switching Channels,1988,PG,04 Mar 1988,105 min,"Comedy, Crime",Ted Kotcheff,"Ben Hecht, Charles MacArthur, Jonathan Reynolds","Kathleen Turner, Burt Reynolds, Christopher Reeve",A television news chief courts his anchorwoman...,...,5.9,"4,033",tt0096203,movie,16 Apr 2012,"$9,129,999",N/A,N/A,True,NaN
9,The House Across the Street,1949,Passed,10 Sep 1949,69 min,"Comedy, Crime, Drama",Richard L. Bare,"Russell S. Hughes, Roy Chanslor","Wayne Morris, Janis Paige, Bruce Bennett","Dave Joslin, the managing editor of a big-city...",...,6.0,349,tt0041486,movie,N/A,N/A,N/A,N/A,True,NaN


In [26]:
missing_fours=four.loc[reem['Title']== 'not found']
print(four.shape)
print(missing_fours.shape)

KeyError: 'Title'

In [ ]:
api=five['api']
year=five['new_year']

api_list=api_pass(api, year)

fives=[]

for x in api_list:
    endpoint = x
    response = requests.get(endpoint)
    if response.status_code != 200:
        print(f'Failed: {response.status_code}')
    else:
        res=response.json()
        fives.append(res)

In [ ]:
fives=pd.json_normalize(fives)
fives